In [79]:
!nvidia-smi

Tue Jan 19 13:29:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   46C    P0    28W /  70W |   1035MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [80]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz
!pip install -q findspark

In [81]:
#LOADING CUDA RAPIDS FOR ACCELERATING THE DISTRIBUTED ML PROCESS TO DECREASE LATENCY

In [82]:
!wget https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar
!wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
!wget https://repo1.maven.org/maven2/ai/rapids/xgboost4j-spark_2.x/1.0.0-Beta5/xgboost4j-spark_2.x-1.0.0-Beta5.jar


--2021-01-19 13:29:47--  https://repo1.maven.org/maven2/ai/rapids/cudf/0.9.2/cudf-0.9.2-cuda10-1.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64759406 (62M) [application/java-archive]
Saving to: ‘cudf-0.9.2-cuda10-1.jar.1’

cudf-0.9.2-cuda10-1 100%[===================>]  61.76M   212MB/s    in 0.3s    

2021-01-19 13:29:47 (212 MB/s) - ‘cudf-0.9.2-cuda10-1.jar.1’ saved [64759406/64759406]

--2021-01-19 13:29:48--  https://repo1.maven.org/maven2/ai/rapids/xgboost4j_2.x/1.0.0-Beta5/xgboost4j_2.x-1.0.0-Beta5.jar
Resolving repo1.maven.org (repo1.maven.org)... 199.232.192.209, 199.232.196.209
Connecting to repo1.maven.org (repo1.maven.org)|199.232.192.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 349225621 (333M) [application/java-archive]
Saving to: ‘xgboost4j_2.x-1.0.0-Beta5.jar.2’



In [83]:
!ls
!pwd

covtype_test.parquet		 spark-3.0.1-bin-hadoop2.7
covtype_train.parquet		 spark-3.0.1-bin-hadoop2.7.tgz
cudf-0.9.2-cuda10-1.jar		 xgboost4j_2.x-1.0.0-Beta5.jar
cudf-0.9.2-cuda10-1.jar.1	 xgboost4j_2.x-1.0.0-Beta5.jar.1
sample_data			 xgboost4j_2.x-1.0.0-Beta5.jar.2
spark-2.4.7-bin-hadoop2.7	 xgboost4j-spark_2.x-1.0.0-Beta5.jar
spark-2.4.7-bin-hadoop2.7.tgz	 xgboost4j-spark_2.x-1.0.0-Beta5.jar.1
spark-2.4.7-bin-hadoop2.7.tgz.1  xgboost4j-spark_2.x-1.0.0-Beta5.jar.2
spark-2.4.7-bin-hadoop2.7.tgz.2
/content


In [84]:
!ls /usr/lib/jvm

default-java		   java-11-openjdk-amd64     java-8-openjdk-amd64
java-1.11.0-openjdk-amd64  java-1.8.0-openjdk-amd64


In [85]:
!pip install -U pyarrow # when we want to bring data frame spark dataframe to pandas data frame overcomes serialization

Requirement already up-to-date: pyarrow in /usr/local/lib/python3.6/dist-packages (2.0.0)


In [86]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

In [87]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /content/cudf-0.9.2-cuda10-1.jar,/content/xgboost4j_2.x-1.0.0-Beta5.jar,/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar pyspark-shell'

In [88]:
import findspark
findspark.init()
findspark.find()
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()
#spark=SparkSession.builder.master("local[*]").getOrCreate()
#spark.conf.set("spark.executor.memory","2g")
#spark.conf.set("spark.driver.memory","2g")
spark.sparkContext.addPyFile('/content/xgboost4j-spark_2.x-1.0.0-Beta5.jar')

In [89]:
import sys,tempfile,urllib
import os
import urllib.request
from pyspark.sql.functions import *
from ml.dmlc.xgboost4j.scala.spark import XGBoostClassificationModel, XGBoostClassifier
from ml.dmlc.xgboost4j.scala.spark.rapids import GpuDataReader
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
import numpy as np
import pandas as pd

In [90]:
from sklearn.datasets import fetch_covtype
from sklearn.datasets import fetch_openml
covtyp = fetch_openml('covertype',version=4)

In [91]:
covtyp.data.shape #54 COLUMNS AND A MILLION RECORDS.

(581012, 54)

In [92]:
np.unique(covtyp.target) #7 classes

array(['1', '2', '3', '4', '5', '6', '7'], dtype=object)

In [93]:
cov_df=pd.DataFrame(data=np.c_[covtyp['data'],covtyp['target']],columns=covtyp['feature_names']+['target'])

In [94]:
cov_df.memory_usage(index=True).sum()

255645408

In [95]:
cov_df.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area1,Wilderness_Area2,Wilderness_Area3,Wilderness_Area4,Soil_Type1,Soil_Type2,Soil_Type3,Soil_Type4,Soil_Type5,Soil_Type6,Soil_Type7,Soil_Type8,Soil_Type9,Soil_Type10,Soil_Type11,Soil_Type12,Soil_Type13,Soil_Type14,Soil_Type15,Soil_Type16,Soil_Type17,Soil_Type18,Soil_Type19,Soil_Type20,Soil_Type21,Soil_Type22,Soil_Type23,Soil_Type24,Soil_Type25,Soil_Type26,Soil_Type27,Soil_Type28,Soil_Type29,Soil_Type30,Soil_Type31,Soil_Type32,Soil_Type33,Soil_Type34,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,target
0,2596,51,3,258,0,510,221,232,148,6279,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,5


In [96]:
print("Rows:",cov_df.shape[0])
print("Columns:",cov_df.shape[1])

Rows: 581012
Columns: 55


In [97]:
cov_df.target.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: target, dtype: int64

In [98]:
cov_df.dtypes

Elevation                             object
Aspect                                object
Slope                                 object
Horizontal_Distance_To_Hydrology      object
Vertical_Distance_To_Hydrology        object
Horizontal_Distance_To_Roadways       object
Hillshade_9am                         object
Hillshade_Noon                        object
Hillshade_3pm                         object
Horizontal_Distance_To_Fire_Points    object
Wilderness_Area1                      object
Wilderness_Area2                      object
Wilderness_Area3                      object
Wilderness_Area4                      object
Soil_Type1                            object
Soil_Type2                            object
Soil_Type3                            object
Soil_Type4                            object
Soil_Type5                            object
Soil_Type6                            object
Soil_Type7                            object
Soil_Type8                            object
Soil_Type9

In [99]:
for cols in cov_df.columns:
    cov_df[cols]=pd.to_numeric(cov_df[cols])

In [100]:
cov_df['target']=cov_df['target']-1 # As machine learning frame work accepts from 0  changing
# 1---7 to 0---6

In [101]:
train_df=cov_df.sample(frac=0.8,random_state=10)
test_df=cov_df.drop(train_df.index)

In [102]:
train_df.to_parquet(fname='covtype_train.parquet',compression='snappy',index=False)
test_df.to_parquet(fname='covtype_test.parquet',compression='snappy',index=False)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: the 'fname'' keyword is deprecated, use 'path' instead
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: the 'fname'' keyword is deprecated, use 'path' instead
  


In [103]:
!ls

covtype_test.parquet		 spark-3.0.1-bin-hadoop2.7
covtype_train.parquet		 spark-3.0.1-bin-hadoop2.7.tgz
cudf-0.9.2-cuda10-1.jar		 xgboost4j_2.x-1.0.0-Beta5.jar
cudf-0.9.2-cuda10-1.jar.1	 xgboost4j_2.x-1.0.0-Beta5.jar.1
sample_data			 xgboost4j_2.x-1.0.0-Beta5.jar.2
spark-2.4.7-bin-hadoop2.7	 xgboost4j-spark_2.x-1.0.0-Beta5.jar
spark-2.4.7-bin-hadoop2.7.tgz	 xgboost4j-spark_2.x-1.0.0-Beta5.jar.1
spark-2.4.7-bin-hadoop2.7.tgz.1  xgboost4j-spark_2.x-1.0.0-Beta5.jar.2
spark-2.4.7-bin-hadoop2.7.tgz.2


In [104]:
import pyarrow.parquet as pq #parquet is a columnar format and lean in size

In [105]:
#To see what is in parquet format we can use pyarrow package

In [106]:
pq.read_table('covtype_train.parquet')

pyarrow.Table
Elevation: double
Aspect: double
Slope: double
Horizontal_Distance_To_Hydrology: double
Vertical_Distance_To_Hydrology: double
Horizontal_Distance_To_Roadways: double
Hillshade_9am: double
Hillshade_Noon: double
Hillshade_3pm: double
Horizontal_Distance_To_Fire_Points: double
Wilderness_Area1: double
Wilderness_Area2: double
Wilderness_Area3: double
Wilderness_Area4: double
Soil_Type1: double
Soil_Type2: double
Soil_Type3: double
Soil_Type4: double
Soil_Type5: double
Soil_Type6: double
Soil_Type7: double
Soil_Type8: double
Soil_Type9: double
Soil_Type10: double
Soil_Type11: double
Soil_Type12: double
Soil_Type13: double
Soil_Type14: double
Soil_Type15: double
Soil_Type16: double
Soil_Type17: double
Soil_Type18: double
Soil_Type19: double
Soil_Type20: double
Soil_Type21: double
Soil_Type22: double
Soil_Type23: double
Soil_Type24: double
Soil_Type25: double
Soil_Type26: double
Soil_Type27: double
Soil_Type28: double
Soil_Type29: double
Soil_Type30: double
Soil_Type31: doubl

In [107]:
#Moving Physical memory into GPU memory
train_data=GpuDataReader(spark).format('parquet').load('covtype_train.parquet')
test_data=GpuDataReader(spark).format('parquet').load('covtype_test.parquet')

In [108]:
train_data.schema

<bound method GpuDataset.schema of <ml.dmlc.xgboost4j.scala.spark.rapids.GpuDataset object at 0x7f9d10cfae48>>

In [109]:
pq_file=pq.read_table('covtype_train.parquet')

In [110]:
pq_file.column_names

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40',
 'target']

In [111]:
label='target'
features=[x for x in pq_file.column_names if x!=label]

In [112]:
features

['Elevation',
 'Aspect',
 'Slope',
 'Horizontal_Distance_To_Hydrology',
 'Vertical_Distance_To_Hydrology',
 'Horizontal_Distance_To_Roadways',
 'Hillshade_9am',
 'Hillshade_Noon',
 'Hillshade_3pm',
 'Horizontal_Distance_To_Fire_Points',
 'Wilderness_Area1',
 'Wilderness_Area2',
 'Wilderness_Area3',
 'Wilderness_Area4',
 'Soil_Type1',
 'Soil_Type2',
 'Soil_Type3',
 'Soil_Type4',
 'Soil_Type5',
 'Soil_Type6',
 'Soil_Type7',
 'Soil_Type8',
 'Soil_Type9',
 'Soil_Type10',
 'Soil_Type11',
 'Soil_Type12',
 'Soil_Type13',
 'Soil_Type14',
 'Soil_Type15',
 'Soil_Type16',
 'Soil_Type17',
 'Soil_Type18',
 'Soil_Type19',
 'Soil_Type20',
 'Soil_Type21',
 'Soil_Type22',
 'Soil_Type23',
 'Soil_Type24',
 'Soil_Type25',
 'Soil_Type26',
 'Soil_Type27',
 'Soil_Type28',
 'Soil_Type29',
 'Soil_Type30',
 'Soil_Type31',
 'Soil_Type32',
 'Soil_Type33',
 'Soil_Type34',
 'Soil_Type35',
 'Soil_Type36',
 'Soil_Type37',
 'Soil_Type38',
 'Soil_Type39',
 'Soil_Type40']

In [113]:
import time
params = { 
    'eta': 0.1,
    'gamma': 0.1,
    'missing': 0.0,
    'treeMethod': 'gpu_hist',
    'maxDepth': 8, 
    'growPolicy': 'depthwise',
    'lambda_': 1.0,
    'subsample': 1.0,
    'numRound': 1000,
    'numWorkers': 1,
    'verbosity': 1
}
classifier = XGBoostClassifier(**params).setLabelCol(label).setFeaturesCols(features)


In [114]:
start_time = time.time()

model=classifier.fit(train_data)

print("GPU Training Time: %s seconds" % (str(time.time() - start_time)))

GPU Training Time: 20.36086654663086 seconds


In [115]:

!nvidia-smi

Tue Jan 19 13:30:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.27.04    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    36W /  70W |   1033MiB / 15079MiB |      9%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [116]:
model.write().overwrite().save('/content/model/')


In [117]:

!ls
#Saved model can be deployed in  production spark to run this model 
# to run in java we can use an option called ml leap
#to run in other languages  convert to pmml file

covtype_test.parquet		 spark-2.4.7-bin-hadoop2.7.tgz.2
covtype_train.parquet		 spark-3.0.1-bin-hadoop2.7
cudf-0.9.2-cuda10-1.jar		 spark-3.0.1-bin-hadoop2.7.tgz
cudf-0.9.2-cuda10-1.jar.1	 xgboost4j_2.x-1.0.0-Beta5.jar
model				 xgboost4j_2.x-1.0.0-Beta5.jar.1
sample_data			 xgboost4j_2.x-1.0.0-Beta5.jar.2
spark-2.4.7-bin-hadoop2.7	 xgboost4j-spark_2.x-1.0.0-Beta5.jar
spark-2.4.7-bin-hadoop2.7.tgz	 xgboost4j-spark_2.x-1.0.0-Beta5.jar.1
spark-2.4.7-bin-hadoop2.7.tgz.1  xgboost4j-spark_2.x-1.0.0-Beta5.jar.2


In [118]:
loaded_model = XGBoostClassificationModel().load('/content/model/')


In [119]:
result=loaded_model.transform(test_data)

In [120]:
result.show()

+---------+------+-----+--------------------------------+------------------------------+-------------------------------+-------------+--------------+-------------+----------------------------------+----------------+----------------+----------------+----------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------+--------------------+--------------------+----------+
|Elevation|Aspect|Slope|Horizontal_Distance_To_Hydrology|Vertical_Distance_To_Hydrology|Horizontal_Distance_To_Roadways|Hillshade_9am|Hillshade_Noon|Hillshade_3pm|Horizontal_Distance_To_Fire_Points|Wild

In [121]:
MulticlassClassificationEvaluator().setLabelCol(label).evaluate(result)

0.7143879104341959